# FIT5196 Assessment 2 Task 1
#### Student Name: Vipul Krishnan Muralee Dharan
#### Student ID: 28104641

Date: 11/05/2018

Version: 1.0

Environment: Python 3.6.1 and Anaconda 4.3.21 (64-bit)

Libraries used:
* pandas v0.20.1 (for dataframe operations)
* re v2.2.1(regular ecpression operations)
* fuzzywuzzy v0.16.0 (for string comparison)
* string (comes with python) (for obtaining all character list)


## 1. Introduction
In this task, we are required to inspect and audit the data (dataset1_with_error.csv) to identify the data problems, and then fix the problems. 

In the end, save the error-free dataset in dataset1_solution.csv. The number of records in your solution should be the same as the number of those in the input file.

## 2. Installing and Importing Libraries

We need the fuzzywuzzy library for this task. We use it in various places for string comparison. Fuzzywuzzy uses another library 'python-Levenshtein' for speeding up its processes (SeatGeek, 2018). If these libraries are not istalled, please install them using the below code and then import all libraries. 

In [1]:
# installing libraries
!pip install python-Levenshtein
!pip install fuzzywuzzy

You are using pip version 9.0.2, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.2, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import string

## 3. Loading and Examining Data
The first step is to load the data from the CSV file.

In [3]:
# loading the data as pandas dataframe
df1 = pd.read_csv("dataset1_with_error.csv")

# checking first 5 rows to see the structure of the data
df1.head()

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,12612628,Engineering Systems Analyst,Dorking,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20121103T000000,20121203T000000
1,12612830,Stress Engineer Glasgow,Glasgow,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130108T150000,20130408T150000
2,12612844,Modelling and simulation analyst,Hampshire,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130726T150000,20130924T150000
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,27504,cv-library.co.uk,20121214T000000,20130314T000000
4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20131025T000000,20131224T000000


Now we perform basic analysis on the data to find out its characteristics. We can use df.info(), df.describe() methods for this.

In [4]:
# displays the information about the dataframe
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25077 entries, 0 to 25076
Data columns (total 11 columns):
Id                  25077 non-null int64
Title               25077 non-null object
Location            25077 non-null object
ContractType        25077 non-null object
ContractTime        25077 non-null object
Company             21242 non-null object
Category            25077 non-null object
Salary per annum    25077 non-null object
SourceName          25077 non-null object
OpenDate            25077 non-null object
CloseDate           25077 non-null object
dtypes: int64(1), object(10)
memory usage: 2.1+ MB


In [5]:
df1.describe()

,Id
count,2.507700e+04
mean,6.664312e+07
std,5.195261e+06
min,1.261263e+07
25%,6.720830e+07
50%,6.836110e+07
75%,6.871371e+07
max,6.924767e+07


The above function describes the newmerical values only. For getting objects, we use it as below.

In [6]:
df1.describe(include=['O'])

,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
count,25077,25077,25077,25077,21242,25077,25077,25077,25077,25077
unique,25077,482,3,3,4879,8,1589,90,2203,2400
top,Graduate Technical Support Analyst Luton,UK,not available,permanent,UKStaffsearch,IT Jobs,35004,totaljobs.com,20130127T150000,20130526T120000
freq,1,3996,19499,16194,248,7085,1011,5335,27,23


We can understand that the 'Title' field is a unique identifier. Category, ContractTime, ContractType has only a limited number of possible values. Other columns have a large number of unique values

Although the task dont ask for checking the missing values, a knowledge of them would help us.

In [7]:
df1.isnull().sum()

Id                     0
Title                  0
Location               0
ContractType           0
ContractTime           0
Company             3835
Category               0
Salary per annum       0
SourceName             0
OpenDate               0
CloseDate              0
dtype: int64

We can see that there are some missing values in the Companu column

##  4. Auditing the Data and Fixing the Problems

Now we can start looking for issues in the data. We can perform a column by column analysis for identifying the problems. We start with the column: 'Id'

### 4a. Column : Id

The Id should be 8 digit and should be unique. This is evident from the data obtained from the describe function. (For referece, it is shown again below.) The maximum value is 6.924767e+07 and the minimum value is 6.924767e+07, which means there are 8 digits in all the Ids. 

In [8]:
# running the describe function again for reference
df1.describe()

,Id
count,2.507700e+04
mean,6.664312e+07
std,5.195261e+06
min,1.261263e+07
25%,6.720830e+07
50%,6.836110e+07
75%,6.871371e+07
max,6.924767e+07


In [9]:
# total number of unique Ids
len(df1.Id.unique())

25077

As shown above, the number of unique values for the Id is equal to the number of rows, which means each vaue is unique.

### 4b. Column: Title

As we seen before each value in the Title column are different. It is the title of the advertised job position. We can check some of the titles.



In [10]:
# to see some of the titles
df1.Title

0                              Engineering Systems Analyst
1                                  Stress Engineer Glasgow
2                         Modelling and simulation analyst
3        Engineering Systems Analyst / Mathematical Mod...
4               Pioneer, Miser Engineering Systems Analyst
5                  Trainee Mortgage Advisor  East Midlands
6                         PROJECT ENGINEER, PHARMACEUTICAL
7        Chef de Partie  Award Winning Restaurant  Exce...
8                                         Quality Engineer
9        Chef de Partie  Award Winning Dining  Live In ...
10            Senior Fatigue and Damage Tolerance Engineer
11                                     C I Design Engineer
12                                 Lead Engineers (Stress)
13         Relief Chef de Partie  Croydon, Surrey  Live in
14             Senior Control and Instrumentation Engineer
15                    Control and Instrumentation Engineer
16                               Electrical / ICA Engine

An analysis shows that the title contains various things such as the actual job position, salary, location etc. It is clear that this is the title of the advertisement.

There are many possible cleanups. For example we can remove the salary specifications, locations etc. from this attribute. 

But it is the decision of the advertising person that what she/he should name it. It reflects a lot of things. For example is the person want to emphasize the salary of the position, she/he puts it on the title. Any other important info can also be added along with the title. For data analytics, these indirect informations can be useful. When we remove these, we are losing these important information. 

So, we can keep this field as entered by the advertiser. However there are some possible issues such as presence of non-ascii characters. We should remove these.

First we look for the non-ascii characters.

In [11]:
# function to look for the non-ascii characters
def test(x):
    # splits into words
    xarray = x.split()
    
    for x1 in xarray:
        # checks non ascii on ach word and prints if it is
        if len(x1) != len(x1.encode()):
            print(x + ' WITH WORD ' + x1)
    return False
# redirecting the output to a dummy variable, we dont want to display
dummy = df1.Title.apply(lambda x: test(x))

Quality Manager – Nursing Homes Scotland ****K WITH WORD –
Support Worker – Domiciliary Care WITH WORD –
Bank Support Worker – Domiciliary Care WITH WORD –
**** **** depending on experience – **** hours flexible shifts required Boston Lincs WITH WORD –
IT Administrator €“ Customer Support, Network and Disaster Recovery A WITH WORD €“
Assistant Manager – Traditional Fish & Chip Shop – Central London WITH WORD –
Assistant Manager – Traditional Fish & Chip Shop – Central London WITH WORD –
Site Technical Manager – Meat Processing WITH WORD –
Chef de Partie – Suffolk  ****AA Rosettes – **** Live In WITH WORD –
Chef de Partie – Suffolk  ****AA Rosettes – **** Live In WITH WORD –
Night Shift Production Manager – Northamptonshire  **** WITH WORD –
Product Technologist – Northamptonshire – **** WITH WORD –
Product Technologist – Northamptonshire – **** WITH WORD –
Shift Manager – Northamptonshire  **** WITH WORD –
QA Manager – Northamptonshire  ****  CRJE****/**** WITH WORD –
Packaging Manager

We can identify the non ascii characters from the above. It is listed in a dictionary with their alternatives in the below code and we remove it

In [12]:
# function for removing non ascii char
def remochar(text, char, new):
    # joinng all characters but changes the non ascii
    return ''.join([i if i != char else new for i in text])

# the dictionary specifying the replacements
dict1 = {'–':'-', '€':'EU ', '“':'"', '’':'\'', '…':'...','—':'-'}

# replacing
for k in dict1.keys():
    df1.loc[:,'Title'] = df1.Title.apply(lambda x: remochar(x,k, dict1[k]))


Now we try checking for non ascii again. We will not find any as we already removed all.

In [13]:
dummy = df1.Title.apply(lambda x: test(x))

### 4c. Column: Location

Location column contains the location of the advertised job position. We can check for the unique values first.

In [14]:
# checking for unique values
locs = list(df1.Location.unique())

In [15]:
# displaying unique values
locs

['Dorking',
 'Glasgow',
 'Hampshire',
 'Surrey',
 'East Midlands',
 'Witney',
 'Derby',
 'Gateshead',
 'UK',
 'Avon',
 'Wolverhampton',
 'Berkshire',
 'North East England',
 'North Yorkshire',
 'East Sheen',
 'Buckinghamshire',
 'London',
 'Central London',
 'York',
 'Newcastle Upon Tyne',
 'Cambridge',
 'Manchester',
 'Maidstone',
 'Bradford',
 'Burgess Hill',
 'Birmingham',
 'Derbyshire',
 'Leicestershire',
 'The City',
 'Staffordshire',
 'Suffolk',
 'Haywards Heath',
 'Jersey',
 'South Yorkshire',
 'Nottinghamshire',
 'Northamptonshire',
 'Milton Keynes',
 'Bristol',
 'Hertfordshire',
 'Basingstoke',
 'Warrington',
 'West Yorkshire',
 'South East London',
 'Salisbury',
 'Leeds',
 'North West London',
 'Poole',
 'Cumbria',
 'Huddersfield',
 'Rotherham',
 'West Sussex',
 'Norwich',
 'Cardiff',
 'Bury St. Edmunds',
 'Braintree',
 'Nottingham',
 'Winchester',
 'Surey',
 'Kent',
 'Durham',
 'Sunderland',
 'Southampton',
 'Essex',
 'Bournemouth',
 'West London',
 'Boston',
 'Leicester',
 

In [16]:
# checking for value counts
df1.Location.value_counts()

UK                      3996
London                  2743
South East London       1433
The City                 558
Leeds                    343
Manchester               334
Surrey                   317
Central London           316
Reading                  300
Birmingham               292
West Midlands            273
East Sheen               222
Bristol                  222
Berkshire                202
Nottingham               196
Oxford                   190
Hampshire                184
Milton Keynes            180
Cambridge                172
Sheffield                160
Kent                     158
Newcastle Upon Tyne      156
Hertfordshire            155
Guildford                147
Aberdeen                 141
Leicester                139
Cheshire                 133
Oxfordshire              127
West Yorkshire           127
Belfast                  125
                        ... 
Cwmbran                    4
Stourbridge                4
Linlithgow                 4
Leads         

There are only 482 values. One of the problem which is seen clearly is the difference in resolution of the location. Sometimes it is country name, sometimes city name and sometimes county name. But it is not proper to consider this as a mistake. For example, if the job is a tourist guide who travels across UK, there is no point in mentioning a particular city. So we are not changing the location.

The words are properly formated, but still there can be some duplicates with spelling mistakes or other small changes. So we compare the string and replace if any. We can use fuzzywuzzy library for this. The library compares the names and returns a weight for the similarity. We use a threshold to filter out the reqired ones. Please not that this threshold can be found using trial and errorr method. 

Before that, we should also remove some meaningless place names such as 'The City' and 'City'. We remove with 'UK' as the all data seems to be from UK

In [17]:
# removing 'City' and 'The City' with 'UK'
df1.Location.replace('City', 'UK', inplace = True)
df1.Location.replace('The City', 'UK', inplace = True)

In [18]:
# making dictionary with key as location and value as count
dict3 = dict(df1.Location.value_counts())

# for each pair of words
for i in dict3.keys():
    for j in dict3.keys():
        # if first letters are same,  the number ration is below a threshold , fuzz ratio more than a threshold and not same
        if  (i[0] == j[0]) & (dict3[i]/dict3[j] < .03) & (fuzz.ratio(i,j) > 70) & (i != j):
            df1.Location.replace('City', 'UK', inplace = True)
            print("Replacing " + i + "  with " + j)

Replacing Reeding  with Reading
Replacing Surey  with Surrey
Replacing Leads  with Leeds
Replacing Oxfords  with Oxford
Replacing Nottinham  with Nottingham


### 4d. Column: ContractType
First we check for the unique values.

In [19]:
# checking for unique values
df1.ContractType.unique()

array(['not available', 'full_time', 'part_time'], dtype=object)

Accodring to the specification this has to be either one of either one of full-time, part-time or non-specified. So we replace with these terms. As it is only 3 values, we can do it manualy but if there are many, we can automate it using fuzzy

In [20]:
# replacing with specified values
df1.ContractType.replace('full_time', 'full-time', inplace = True)
df1.ContractType.replace('part_time', 'part-time', inplace = True)
df1.ContractType.replace('not available', 'non-specified', inplace = True)

### 4e. Column: ContractTime
We check for unique values

In [21]:
# checking for unique values
df1.ContractTime.unique()

array(['permanent', 'not available', 'contract'], dtype=object)

Similat to the previous one, we replace with the specified values

In [22]:
# replacing with specified values
df1.ContractTime.replace('not available', 'non-specified', inplace = True)

### 4f. Column: Company

First we check for the unique values

In [23]:
# obtaining the unique values for company
comp = df1.Company.unique()

In [24]:
# trying to print
comp

array(['Gregory Martin International', 'Brite Recruitment',
       'MatchBox Recruiting Ltd', ..., 'Finegreen Associates',
       'Orion Business Management', 'Gleeson Accountancy Recruitment Ltd'], dtype=object)

In [25]:
# checking the number
len(comp)

4880

In [26]:
df1.Company.value_counts()

UKStaffsearch                            248
CVbrowser                                170
Randstad                                 169
JOBG8                                    159
Matchtech Group plc.                     150
Chef Results                             137
Penguin Recruitment                      134
Hays                                     115
London4Jobs                              112
COREcruitment International              111
JAM Recruitment Ltd                      109
Computer People                           99
Clear Selection                           92
Monarch Recruitment                       89
Towngate Personnel                        79
Cherryred Recruitment                     69
Senitor Associates                        67
Capita Resourcing                         62
Support Services Group                    62
Modis                                     62
Aspire Data Recruitment                   61
Adecco                                    57
Evolution 

There are 4880 unique values. We have also found that there are some missing values. Although we are not ased to remove missing values, for easiness of processing, we fill the missing values with 'not available'.

In [27]:
# filling missing values with not available
df1.Company.fillna('not available', inplace = True)

Now we check for some junk values

In [28]:
# checking for values with length less than 2
df1[df1.Company.apply(lambda x: len(x) < 2)]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
20472,68849372,Apply Today Start Tomorrow,UK,non-specified,permanent,.,"PR, Advertising & Marketing Jobs",18000,jobstoday.co.uk,20130701T120000,20130830T120000


There is only a dot here. We just conider it as a missing value and remove with 'not available'

In [29]:
df1.Company.replace('.','not available', inplace = True)

There are some inconsistency in data too. The same names are written in different forms. For example some company names are written with and without 'Limited' and sometime they use 'Ltd'. Now we look for these incosistencies and also try to make data in all same format. (eg all first letters in capital and all other in small). 

makenice() function formats the data (as all first letters in capital and all other in small), and also removes the ambuiguity of limited and ltd.

checkformatch() function is used to check is two names are equivalent. More precisely it checks if one name is completely inlcuded in the other name. (ie each word in one name is there in the other name). Some trial and error exercise shows that this is much useful than the fuzz to compare the company names..

In [30]:
# function to clean each name
def makenice(line):
    # takes each word
    words = line.strip().split()
    newline = ''
    # re-makes the name with cleaned words
    # this for loop takes each word, capitalize the first char, and strips any unwanted
    for w in words:
        w = w.strip().strip('.').lower().strip()
        if w != '':
            w = w[0].upper()+w[1:] if len(w) > 1 else w[0].upper()
            # checks for the word Limited and convers to Ltd for uniformity
            if w=='Limited':
                w = 'Ltd'
        newline = newline + ' ' + w
    # returns the cleaned name
    return newline.strip(' ')


In [31]:
# unifying the format of the written names
df1.loc[:,'Company'] = df1.Company.apply(lambda y: makenice(y))

checkformatch() function is used to check is two names are equivalent. More precisely it checks if one name is completely inlcuded in the other name. (ie each word in one name is there in the other name). Some trial and error exercise shows that this is much useful than the fuzz to compare the company names.

autocorrect function identifies the same companies written as different and unifies.

Please note that it will unify only certain conditions.

If there is a name x and name y and y is x + something else (eg. x='coles' and y = 'coles group') and there exists only a single y (ie for x = coles, there is no two y such as 'coles petrol' and 'coles supermarkets')

This is to ensure that we dont group together entirely different companies. Also we replace a name with longer name only, ie we dont generalize something like Coles Petrols and Coles Supermarkets as Coles. This can often make problems. (eg, Most of the companies under the name Mitsubushi are entirely unrelated firms (Wikipedia, 2018)

We also check that the replacing company names have same first word. Most often, the first words remain unchanged even if we write company names differently.

In short we try to be safe while cleaning up the data.

In [32]:
# check for matches between names
def checkformatch(c1, c2):
    # takes each words of both names
    c1l = c1.split()
    c2l = c2.split()
    
    # checks is each word in c1 in c1
    f1 = True
    for w in c1l:
        if w not in c2l:
            f1 = False
    f2 = True
    
    # checks if each word in c2 in c1
    for w in c2l:
        if w not in c1l:
            f2 = False
            
    # if any of the above is true, returns true
    return (f1 | f2)

# autocorrect inconsistencies
def autocorrect(letter):
    # select all unique company names with the given letter
    comp = list(df1[df1.Company.apply(lambda x: (x[0] == letter))].Company.unique())
    
    # dictionary to group together the names
    dict2 = {}
    
    for c1 in comp:
        for c2 in comp:
            # compare 2 names and if they matches and the first words are same
            if checkformatch(c1, c2) & (c1 != c2) & (c1.split()[0] == c1.split()[0]):
                # if it is jsut a difference of missing 'Ltd's, just append it 
                if (c1 + ' Ltd' == c2):
                    df1.Company.replace(c1, c2)
                elif (c2 + ' Ltd' == c1):
                    df1.Company.replace(c2, c1)
                
                else:
                    count1 = len(df1[df1.Company == c1])
                    count2 = len(df1[df1.Company == c2])
                    
                    # arranging the pair as [shorter, longer]
                    sizearr = [c1,c2] if len(c1)<len(c2) else [c2,c1]
                    
                    # for each shorter we make a list of possible longer names and add to dictionary
                    if sizearr[0] in dict2.keys():
                        dict2[sizearr[0]].append(sizearr[1])
                    else:
                        dict2[sizearr[0]] = [sizearr[1],]
        # for increasing the speed            
        comp.remove(c1)
    # count for replacements in this execution
    i = 0
    # for each smaller name
    for w in dict2.keys():
        # if there is only possible longer name, replace it, increase count and print
        if len(dict2[w]) == 1:
            i = i + 1
            df1.Company.replace(w,dict2[w][0], inplace=True)
            print(w + ' replaced with ' + dict2[w][0])
    # return the count
    return i

Now we run autocorrect function for each letter. The comparison is limited by the letter as otherwise the execution takes too much time. We use a while loop to repeatedly execute the function because there can be cascaded relations such as 'Coles', 'Coles Group', 'Coles Group Ltd.'. For this, in the first execution Coles becomes Coles Group and then in next all Coles Group becomes Coles Group Ltd. We repeat until we finds that there is no more changes happening.

In [33]:
# obtaining all characters 
allchar = string.ascii_uppercase + string.digits + string.punctuation

# repeated execution of the autocorrect function
while(True):
    count = 0
    # for each possible first character, execute funtion 
    for let in allchar:
        count = count + autocorrect(let)
    # repeat untill the retuened count of an execution turns zero
    # there can be cascaded names such as 'Coles', 'Coles Group', 'Coles Group Ltd.'
    # for this, in the first execution Coles becomes Coles Group and then in next all Coles Group becomes Coles Group Ltd.
    # This is why we need this repeted calling of function
    if count == 0:
        break

Amrec replaced with Amrec Recruitment Ltd
Asquith Nurseries replaced with Asquith Day Nurseries
Allied Healthcare replaced with Allied Healthcare Group
Antal International replaced with Antal International Ltd Warrington
Alderwood replaced with Alderwood Education
Appetite4recruitment High End Restaurants, Pubs Bars replaced with Appetite4recruitment High End Restaurants, Pubs & Bars
Amdas replaced with Amdas Consultancy Ltd
Ad Lib replaced with Ad Lib Holdings Ltd
Avenue Scotland replaced with Avenue Scotland Falkirk
Acceler It replaced with Acceler It Recruitment Consultancy
Ambridge Harris replaced with Ambridge Harris Associates
Astute Ltd replaced with Astute Technical Recruitment Ltd
Astute Technical replaced with Astute Technical Recruitment Ltd
Astute Recruitment replaced with Astute Technical Recruitment Ltd
Appetite4recruitment Contract Facilities Division replaced with Appetite4recruitment Contract & Facilities Division
Arrows Group replaced with Arrows Group Healthcare
Admi

Swiis replaced with Swiis Healthcare
Sykes Co Recruitment replaced with Sykes & Co Recruitment Ltd
Smart Recruitment Solutions replaced with Smart Solutions Recruitment
Ser Ltd replaced with Ser Staffing Ltd
Syk Recruitment replaced with Syk Recruitment Specialists
Str replaced with Str Health
Ssp replaced with Ssp Uk Ltd
Synergy replaced with Synergy Health Recruitment
Search Recruitment replaced with Search Recruitment Services Ltd
Sarah Harvey replaced with Sarah Harvey Recruitment
Seven Steps replaced with Seven Steps Recruitment Ltd
Steps Recruitment replaced with Seven Steps Recruitment Ltd
Sales Recruitment Services replaced with Sales Recruitment Services Srs Uk Ltd
Spire replaced with Spire Healthcare
Stott May replaced with Stott & May
Sw6 replaced with Sw6 Associates
Sanderson Recruitment replaced with Sanderson Recruitment Plc
Spargonet Consulting replaced with Spargonet Consulting Plc
Sellick Partnership replaced with Sellick Partnership Midlands
Scom replaced with Scom Ma

### 4g. Column: Category

We check the unique values

In [34]:
# checking unique values
df1.Category.unique()

array(['Engineering Jobs', 'Accounting & Finance Jobs',
       'Healthcare & Nursing Jobs', 'Hospitality & Catering Jobs',
       'IT Jobs', 'Sales Jobs', 'Teaching Jobs',
       'PR, Advertising & Marketing Jobs'], dtype=object)

There are only some values and all the values are proper. So we cant find any issue with this attribute.

### 4h. Column: Salary per annum
The salary per annum should be a number. We check for non numbers

In [35]:
# checking for non digit salary values
df1[df1['Salary per annum'].apply(lambda x: not str(x).isdigit())]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
133,46626860,Registered Home Manager Job Bournemouth,Bournemouth,full-time,non-specified,Not Available,Healthcare & Nursing Jobs,30K,careworx.co.uk,20130329T150000,20130528T150000
238,46627928,Care Home Manager Job North London ****K,London,full-time,non-specified,Not Available,Healthcare & Nursing Jobs,38K,careworx.co.uk,20121107T150000,20130205T150000
305,46628805,Home Care Workers Berkhampsted,UK,part-time,non-specified,Not Available,Healthcare & Nursing Jobs,14K,careworx.co.uk,20120905T150000,20121104T150000
596,46634306,Home Manager Mental Health 11 bed,Wales,non-specified,non-specified,Not Available,Healthcare & Nursing Jobs,24K,careworx.co.uk,20120913T120000,20121112T120000
647,46634923,RGN Nurse Hull Days or Nights **** per hour,UK,full-time,non-specified,Not Available,Healthcare & Nursing Jobs,20896.2 - 23095.8,careworx.co.uk,20130605T150000,20130903T150000
830,46637596,Staff Nurse South Shields ****,South Shields,non-specified,non-specified,Not Available,Healthcare & Nursing Jobs,23K,careworx.co.uk,20120429T150000,20120513T150000
896,48082563,Senior Chef de Partie One AA Rosette Hotel T...,Cumbria,non-specified,non-specified,Chef Results,Hospitality & Catering Jobs,16153.8 - 17854.2,caterer.com,20130109T150000,20130123T150000
952,49065458,Service Manager Learning Disabilities,Wales,non-specified,non-specified,Not Available,Healthcare & Nursing Jobs,25K,careworx.co.uk,20130108T150000,20130122T150000
980,49689021,Assessment Officer,Kent,non-specified,non-specified,Not Available,Healthcare & Nursing Jobs,23712.0 - 26208.0,careworx.co.uk,20120220T000000,20120321T000000
996,49845058,Chef de Partie Fresh Food Pub Good Reputatio...,Lancashire,non-specified,non-specified,Chef Results,Hospitality & Catering Jobs,16K,caterer.com,20120601T150000,20120701T150000


We find two kinds of problmes. Some times salary is given as a range and some times 'K' is used to represent 1000.

We correct the first one by using the average value and second one by representing the number in proper form

In [36]:
# removing 'K' in salary with '000'
df1.loc[:,'Salary per annum'] = df1['Salary per annum'].apply(lambda x: float(x.strip('K').strip('k')+'000') if ('k' in x) | ('K' in x) else x)

In [37]:
# changing the range to avg value
df1.loc[:,'Salary per annum'] = df1['Salary per annum'].apply(lambda x: round((float(str(x).split('-')[0].strip())+float(str(x).split('-')[0].strip()))/2) if ('-' in str(x)) else x)

### 4i. Column: Source Name

Accordning to the specification, the source name should be a website. We check if all satidfies this. We use python re for this

In [38]:
# viewing all source names which are not websites
df1[df1.SourceName.apply(lambda x: re.match(r'[0-9a-zA-Z\.\-]+\.[a-zA-Z]+$', x) == None)]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
5663,66932999,Registered Midwives RM Lincolnshire Lincoln,Lincoln,part-time,non-specified,The A24 Group,Healthcare & Nursing Jobs,40140,monashstudent,20130124T150000,20130223T150000
12871,68393935,Killer Javascript role Exclusive to Brightwater,Belfast,full-time,permanent,Brightwater Group,IT Jobs,39996,jobcareer,20120205T150000,20120306T150000
15379,68672352,Digital Account Manager **** plus a Great Bonus,South East London,non-specified,permanent,Blu Digital,Sales Jobs,27504,admin@caterer.com,20120516T000000,20120715T000000


There are only 3 entries. We assume that the 'monashstudent' value refers to Monash students career portal, jobcareer refers to jobcareer.com which is a real career website and use caterer.com instead of admin@caterer.com

In [39]:
# replacing invalid sources
df1.SourceName.replace('monashstudent', 'careergateway.monash.edu.au', inplace = True)
df1.SourceName.replace('jobcareer', 'jobcareer.com', inplace = True)
df1.SourceName.replace('admin@caterer.com', 'caterer.com', inplace = True)

### 4j. Column: OpenDate

This field is a date. We can validate the date to find if all the entries are valid dates. We can use pyhton re for this

In [40]:
# printing invalid dates in OpenDate
df1[df1.OpenDate.apply(lambda x: re.match(r'201[0-9]((((0[13578]0[1-9]T.{6})|(0[13578][12][0-9]T.{6})|(0[13578]3[0-1]T.{6}))|((020[1-9]T.{6})|(02[12][0-9]T.{6}))|((0[4,6,9]0[1-9]T.{6})|(0[4,6,9][12][0-9]T.{6})|(0[4,6,9]30T.{6})))|(((1[02]0[1-9]T.{6})|(1[02][12][0-9]T.{6})|(1[02]3[0-1]T.{6}))|((110[1-9]T.{6})|(11[1-2][0-9]T.{6})|(1130T.{6}))))', x)==None)]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
1102,51593546,Community Nurse South London,UK,non-specified,non-specified,Not Available,Healthcare & Nursing Jobs,29004,careworx.co.uk,20131803T120000,20130417T120000
2104,61282006,Deputy Home Care Manager,Barnet,non-specified,non-specified,Not Available,Healthcare & Nursing Jobs,20004,careworx.co.uk,20132606T000000,20130710T000000
2839,64796836,"Technical Tester Manual, Automation, Java, SQ...",London,non-specified,permanent,Not Available,IT Jobs,19500,theitjobboard.co.uk,20122003T150000,20120519T150000
5707,66935867,"Technical Sales Manager, Pigments Fillers to S...",UK,non-specified,non-specified,Wallace Hind Selection Llp,Sales Jobs,45000,salestarget.co.uk,20121512T150000,20130213T150000
10881,68257270,Desire a Change? London ****K Basic ****K OTE,South East London,non-specified,non-specified,Zest 2 Recruitment,Sales Jobs,13500,salestarget.co.uk,20133004T150000,20130530T150000
11948,68350622,Mechanical Design Engineer (HVAC),Gainsborough,non-specified,permanent,Not Available,IT Jobs,39996,technojobs.co.uk,20131908T000000,20131117T000000
15353,68671948,Investment Analyst,Cheshire,non-specified,permanent,Idex Consulting Llp,Accounting & Finance Jobs,24000,totaljobs.com,20121406T120000,20120912T120000
22918,69072305,Scheme Actuary,Hampshire,non-specified,permanent,Sammons Pensions,Accounting & Finance Jobs,54996,professionalpensionsjobs.com,20131509T000000,20131114T000000
23007,69079402,Naval architect/structural engineer,Newcastle Upon Tyne,non-specified,permanent,Workshop Recruitment,Engineering Jobs,33504,cv-library.co.uk,20132901T150000,20130429T150000
23169,69080910,MS (Elec biased) Maintenance Engineer / Superv...,Harrogate,non-specified,permanent,Jenrick Engineering,Engineering Jobs,17004,cv-library.co.uk,20132108T120000,20131119T120000


We can find that in the above dates, the month number is greater than 12. But at the same time the day value is all less than 12. This could be an arror caused because of some sources following different date representation format such as US date format.

We swap the month and date mutually.

In [41]:
# correcting invalid open dates
df1.loc[:,'OpenDate'] = df1.OpenDate.apply(lambda x: (x[0:4] + x[6:8] + x[4:6] + x[8:]) if (int(x[4:6]) > 12) else x )

### 4k. Column: CloseDate

We check for invalid dates in this field also.

In [42]:
# printing invalid dates in CloseDate
df1[df1.CloseDate.apply(lambda x: re.match(r'201[0-9]((((0[13578]0[1-9]T.{6})|(0[13578][12][0-9]T.{6})|(0[13578]3[0-1]T.{6}))|((020[1-9]T.{6})|(02[12][0-9]T.{6}))|((0[4,6,9]0[1-9]T.{6})|(0[4,6,9][12][0-9]T.{6})|(0[4,6,9]30T.{6})))|(((1[02]0[1-9]T.{6})|(1[02][12][0-9]T.{6})|(1[02]3[0-1]T.{6}))|((110[1-9]T.{6})|(11[1-2][0-9]T.{6})|(1130T.{6}))))', x)==None)]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate


So all the dates are valid. One of the other possible issue is ClosingDate being before the OpenDate. We check for it also.

In [43]:
# checking for opendates > closedate
df1[df1.apply(lambda x: int(x.OpenDate[0:8]) > int(x.CloseDate[0:8]), axis = 1)]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
6659,67330693,Pensions Implementation Manager,Brighton,non-specified,permanent,Not Available,Accounting & Finance Jobs,35004,hays.co.uk,20130703T000000,20130404T000000
9568,68091856,Manager Technical Operations,Shepton Mallet,non-specified,permanent,Index Recruitment,IT Jobs,62004,totaljobs.com,20120306T000000,20120205T000000
11043,68290941,Planning Enforcement officer,London,full-time,contract,Randstad,Engineering Jobs,39360,jobs.planningresource.co.uk,20131105T000000,20131006T000000
12473,68360715,Penetration Tester / Reverse Engineer,UK,non-specified,contract,Vertex Solutions,IT Jobs,76908,cwjobs.co.uk,20131222T150000,20131023T150000
19142,68729089,Registered Home Manager Loughborough,Loughborough,full-time,non-specified,Compass Associates Ltd,Healthcare & Nursing Jobs,42504,staffnurse.com,20120803T150000,20120505T150000
24206,69173205,"User Interface/UI Developer JavaScript, jQuer...",London,non-specified,permanent,Dawson & Walsh,IT Jobs,45000,totaljobs.com,20130622T150000,20130324T150000
24297,69181738,Registered General Nurse (RGN)/ Registered Men...,Poole,full-time,non-specified,Executive Care Group Ltd,Healthcare & Nursing Jobs,24960,staffnurse.com,20130116T000000,20121018T000000
25039,69228213,"SCIENTIFIC (PHYSICS, OR CHEMISTRY, OR ELECTRON...",Warrington,full-time,permanent,Not Available,Engineering Jobs,24996,fish4.co.uk,20130708T120000,20130608T120000


We assume that this hapended because the dates get swapped. So we swap again to solve this problem.

In [44]:
# storing corrected dates serieses in seperate variables
newOpenDate = df1.apply(lambda x: x.CloseDate if (int(x.OpenDate[0:8]) > int(x.CloseDate[0:8])) else x.OpenDate, axis = 1)
newCloseDate = df1.apply(lambda x: x.OpenDate if (int(x.OpenDate[0:8]) > int(x.CloseDate[0:8])) else x.CloseDate, axis = 1)

# substituting stored corrected serieses to dataframe
df1.loc[: , 'OpenDate'] = newOpenDate
df1.loc[: , 'CloseDate'] = newCloseDate

## 5. Storing the Data 

Now we have the corrected dataset. We store it in the file as mentioned in the question

In [45]:
# storing data into file
df1.to_csv("dataset1_solution.csv", encoding = 'utf-8', index=False)

## 6. Summary
The given data contained a lot of errors. We have done the following works to clean it.
* removed Non-ascii characters from Tilte
* corrected spelling mistakes in Location
* replaced with proper phrases in ContractType and ContractTime
* formatted company names and joined together similar companies whenever possible
* reformatted and made the Salary per annum representation uniform
* corrected invalid dates from OpenDate
* corrected the problem of CloseDate being earlier than OpenDate

After the cleanup, the data is stored as csv file.

## References

- SeatGeek. (2018). 'FuzzyWuzzy'. Retrieved from https://github.com/seatgeek/fuzzywuzzy
- Wikipedia. (2018). 'Mitsubishi'. Retrieved from https://en.wikipedia.org/wiki/Mitsubishi
